The NOAA HRRR is a real-time 3km resolution, hourly updated, cloud-resolving, convection-allowing atmospheric model, initialized by 3km grids with 3km radar assimilation.

This notebook provides an example of accessing HRRR data, including (1) finding the data file corresponding to a date and time, (2) retrieving a portion of that file from blob storage which includes the surface temperature variable, (3) opening the file using the xarray library, and (4) rendering an image of the forecast.

This dataset is documented at http://aka.ms/ai4edata-hrrr.

In [2]:
import climatedata_functions as climf
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
metadata = pd.read_csv("../data/metadata.csv")

In [4]:
ds = climf.get_ds()#only words with recent dates

### Make metadafile with gridpoints

In [5]:
#tranform coordinates for longitude
metadata['longitude_trans'] = metadata['longitude']+360 
#add columns for gridpoints
metadata['x_grid'] = ''
metadata['y_grid'] = ''

#make new metadatafile and save as cvs 
#metadata_new = climf.save_grids(metadata.head(), ds) #(takes approx 6 minutes)
#metadata_new.to_csv('../data/metadata_grids.csv')


In [6]:
#load new metadata file that includes the gridpoints
metadata = pd.read_csv("../data/metadata_grids.csv")

In [7]:
metadata.head(10)

,Unnamed: 0,uid,latitude,longitude,date,split,longitude_trans,x_grid,y_grid
0,0,aabm,39.080319,-86.430867,2018-05-14,train,273.569133,570,1217
1,1,aabn,36.559700,-121.510000,2016-08-31,test,238.490000,550,192
2,2,aacd,35.875083,-78.878434,2020-11-19,train,281.121566,488,1455
3,3,aaee,35.487000,-79.062133,2016-08-24,train,280.937867,473,1453
4,4,aaff,38.049471,-99.827001,2019-07-23,train,260.172999,513,831
5,5,aafl,39.474744,-86.898353,2021-08-23,train,273.101647,583,1202
6,6,aafp,35.647742,-79.271782,2017-11-15,train,280.728218,478,1445
7,7,aagm,35.906885,-79.132962,2020-06-10,train,280.867038,488,1447
8,8,aahy,35.726522,-79.125458,2014-08-12,train,280.874542,481,1449
9,9,aaia,35.980000,-78.791686,2018-06-27,train,281.208314,493,1457


### Get temperatures for gridpoints

Make function that gets the mean/median temperature for one date for all the places sampled at that date (to save time) and the previos x days

In [8]:
metadata['temp'] = ''

In [9]:
from datetime import timedelta


start_date = climf.get_start_date(metadata.date[0])
date_index_list = metadata.index[metadata.date == start_date]
#print(date_index_list)
temp_list = []
done_list = []
days = 7

for idx,row in enumerate(metadata.index): #takes first element in metadata list
    #print(row, idx)
    start_date = metadata.date[idx]
    date_index_list = metadata.index[metadata.date == start_date]#list with all the indexes of dates with the same date
    #print(date_index_list)
    if start_date not in done_list:#only do if this date wasnt used before
        temp_list = [[0] * days for i in range(len(date_index_list))]#make list of lists to store the values inside
        #print(len(temp_list), len(temp_list[0]))
        done_list.append(start_date)#list of dates already samples
        start_date = climf.get_start_date(start_date)#formate to time object
        print(start_date)
        for x in range(days):
            count = 0
            day_date = start_date - timedelta(days=x)
            ds, stop = climf.get_ds_aws(day_date)#getting the temperature array for the specified date
            for index in date_index_list:
                x_grid = metadata.x_grid[index]
                y_grid = metadata.y_grid[index]
                if stop == True:
                    temp_list[count][x] = 'na'
                else:
                    temp_list[count][x] = ds[x_grid][y_grid]
                count += 1
                if x == days-1 and index == date_index_list[len(date_index_list)-1]:#if condition is met put the values in the metadata file
                    for i in range(len(temp_list)):                    
                        metadata.temp.loc[date_index_list[i]] = temp_list[i] #not index but 
        if idx == 5:#just for test
            break  
    else:
        continue 

2018-05-14


/Users/janataumer/neue_fische/tick-tick-bloom/.venv/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


2016-08-31


/Users/janataumer/neue_fische/tick-tick-bloom/.venv/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


2020-11-19


...

/Users/janataumer/neue_fische/tick-tick-bloom/.venv/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


2019-07-23


/Users/janataumer/neue_fische/tick-tick-bloom/.venv/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


2021-08-23


/Users/janataumer/neue_fische/tick-tick-bloom/.venv/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [10]:
metadata.head()

,Unnamed: 0,uid,latitude,longitude,date,split,longitude_trans,x_grid,y_grid,temp
0,0,aabm,39.080319,-86.430867,2018-05-14,train,273.569133,570,1217,"[287.3599, 287.36548, 287.3154, 287.20593, 287..."
1,1,aabn,36.559700,-121.510000,2016-08-31,test,238.490000,550,192,"[296.57178, 293.3236, 291.91608, 292.43445, na..."
2,2,aacd,35.875083,-78.878434,2020-11-19,train,281.121566,488,1455,"[274.84113, 279.1203, 279.9865, 292.71667, 283..."
3,3,aaee,35.487000,-79.062133,2016-08-24,train,280.937867,473,1453,"[294.81213, 295.3043, 297.76993, 300.13126, 29..."
4,4,aaff,38.049471,-99.827001,2019-07-23,train,260.172999,513,831,"[297.67825, 300.68784, 305.18024, 305.14374, 3..."


In [13]:
(metadata.temp != '').count()

23570

In [16]:
metadata[metadata.temp != ''].count()

Unnamed: 0         263
uid                263
latitude           263
longitude          263
date               263
split              263
longitude_trans    263
x_grid             263
y_grid             263
temp               263
dtype: int64